In [307]:
import pandas as pd
import geopandas as gp
from pathlib import Path
import numpy as np
import math
import itertools
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
def replace_region(name):
    try:
        name = name.replace('MIMAROPA REGION',"REGION IV-B (MIMAROPA)")
        name = name.replace('REGION III - CENTRAL LUZON',"REGION III (Central Luzon)")
        name = name.replace('REGION II - CAGAYAN VALLEY',"REGION II (Cagayan Valley)")
        name = name.replace('REGION VIII - EASTERN VISAYAS',"REGION VIII (Eastern Visayas)")
        name = name.replace('REGION I - ILOCOS',"REGION I (Ilocos Region)")
        name = name.replace('REGION IV-A - CALABARZON',"REGION IV-A (CALABARZON)")
        name = name.replace('CORDILLERA ADMINISTRATIVE REGION',"CAR - Cordillera Administrative Region")
        name = name.replace('REGION VI - WESTERN VISAYAS',"REGION VI (Western Visayas)")
        name = name.replace('AUTONOMOUS REGION IN MUSLIM MINDANAO',"BARMM - Bangsamoro Autonomous Region in Muslim Mindanao")
        name = name.replace('REGION XII - SOCCSKSARGEN',"REGION XII (Soccsksargen)")
        name = name.replace('REGION VII - CENTRAL VISAYAS',"REGION VII (Central Visayas)")
        name = name.replace('REGION XIII - CARAGA',"REGION XIII (Caraga)")
        name = name.replace('REGION IX - ZAMBOANGA PENINSULA',"REGION IX (Zamboanga Peninsula)")
        name = name.replace('REGION X - NORTHERN MINDANAO',"REGION X (Northern Mindanao)")
        name = name.replace('REGION V - BICOL',"REGION V (Bicol Region)")
        name = name.replace('REGION XI - DAVAO',"REGION XI (Davao Region)")
        name = name.replace('NATIONAL CAPITAL REGION',"NCR - National Capital Region")
        return name
    except:
        return name 

In [308]:
# Setting up directories

WORKINGDIR = Path(os.getcwd())
PROJECTROOT = WORKINGDIR.parents[1]
DATASET = Path(PROJECTROOT,"data","gathered-datasets","housing-census","housing-census-housetype-2015.csv")

In [309]:
df = pd.read_csv(DATASET)
df = df.iloc[:, :7] # Take only first 7 columns

In [310]:
df.head()

,Type of Building and City/Municipality,Occupied Housing Units,Number of Households*,Household Population*,Average Household Size,Ratio of Households to Occupied Housing Units,Ratio of Household Population to Occupied Housing Units
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NATIONAL CAPITAL REGION,NaN,NaN,NaN,NaN,NaN,NaN
2,Total,2.968.651,3.095.484,12.786.611,"4,13","1,04","4,31"
3,Single house,1.333.067,1.396.332,6.378.542,"4,57","1,05","4,78"
4,Duplex,410.775,427.606,1.733.813,"4,05","1,04","4,22"


In [311]:
df = df.loc[df["Type of Building and City/Municipality"]!="Total"]

In [312]:
# Group into regions
df['group'] = df.isnull().all(axis=1).cumsum() 

In [313]:
regions = list(df["group"].unique())

In [314]:
df.head()

,Type of Building and City/Municipality,Occupied Housing Units,Number of Households*,Household Population*,Average Household Size,Ratio of Households to Occupied Housing Units,Ratio of Household Population to Occupied Housing Units,group
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NATIONAL CAPITAL REGION,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Single house,1.333.067,1.396.332,6.378.542,"4,57","1,05","4,78",1
4,Duplex,410.775,427.606,1.733.813,"4,05","1,04","4,22",1
5,Multi-unit residential,1.207.374,1.252.815,4.608.041,"3,68","1,04","3,82",1


In [315]:
df["Type of Building and City/Municipality"] = df["Type of Building and City/Municipality"].apply(replace_region)

In [316]:
housing_stats = [' Occupied Housing Units ',
       ' Number of Households* ', ' Household Population* ',
       ' Average Household Size ',
       ' Ratio of Households to Occupied Housing Units ',
       ' Ratio of Household Population to Occupied Housing Units ']

In [317]:
bldg_types = ['Single house', 'Duplex',
       'Multi-unit residential', 'Commercial/industrial/agricultural',
       'Institutional living quarter', 'Others', 'Not Reported',]

In [318]:
all_vals = []

In [319]:
def parse_region(df):
    df = df.copy()
    region_name = df["Type of Building and City/Municipality"].loc[1]
    unique_vals = df["Type of Building and City/Municipality"].dropna().unique()
    cities = [value for value in unique_vals if value not in bldg_types and value != region_name]

    for city in cities:
        print(f">>> Parsing {city} in {region_name}...")
        idx = (df_.loc[df_["Type of Building and City/Municipality"]==city].index).tolist()[0]
        city_vals_list = parse_city(city.strip(), region_name, df, idx)
        
        for row in city_vals_list:
            all_vals.append(row)

In [320]:
def parse_city(city, region_name, df, idx):
    df = df[idx:idx+8].copy()
    
    city_vals_list = []
    
    for bldg_type in bldg_types:
        city_vals = {}
        city_vals["city"]=city
        city_vals["region_name"]=region_name
        city_vals["bldg_type"]=bldg_type
        
        for housing_stat in housing_stats:
            

            df_ = df.loc[df["Type of Building and City/Municipality"]==bldg_type]
            
            replace_period = [' Occupied Housing Units ', ' Number of Households* ', ' Household Population* ']
            
            if housing_stat in replace_period:
                val = df_[housing_stat].values
                val = str(val[0]).strip()
                val = val.replace(".","")
                
            else: 
                val = df_[housing_stat].values
                val = str(val[0]).strip()
                val = val.replace(",",".")
            
            try:    
                city_vals[housing_stat.strip()] = float(val)      
            except:
                city_vals[housing_stat.strip()] = float(np.nan)   
                
            city_vals_list.append(city_vals)
    
    return city_vals_list 
    

In [324]:
def derive_province_from_region(df):
    df = df.copy()
    region_name = df["Type of Building and City/Municipality"].loc[1]
    unique_vals = df["Type of Building and City/Municipality"].dropna().unique()
    cities = [value for value in unique_vals if value not in bldg_types and value != region_name]

    print(region_name)
    possible_provinces = list(ref_df["province"].loc[ref_df["region"]==region_name].unique())
    print(possible_provinces)
    
    matched_provinces = []
    
    for possible_province in possible_provinces:
        for city in cities:
            if possible_province.lower() in city.lower() and "city" not in possible_province.lower():
                matched_provinces.append(city)

In [325]:
all_matched = []

In [326]:
for region in regions:
    df_ = df.loc[df["group"]==region].reset_index()
    matched_provinces = derive_province_from_region(df_)
    
    all_matched.append(matched_provinces)

NCR - National Capital Region
['Metropolitan Manila']
CAR - Cordillera Administrative Region
['Ifugao', 'Benguet', 'Kalinga', 'Abra', 'Mountain Province', 'Apayao']
REGION I (Ilocos Region)
['Ilocos Norte', 'Pangasinan', 'La Union', 'Ilocos Sur']
REGION II (Cagayan Valley)
['Cagayan', 'Quirino', 'Nueva Vizcaya', 'Isabela', 'Batanes']
REGION III (Central Luzon)
['Bataan', 'Nueva Ecija', 'Tarlac', 'Bulacan', 'Pampanga', 'Aurora', 'Zambales']
REGION IV-A (CALABARZON)
['Quezon', 'Batangas', 'Laguna', 'Cavite', 'Rizal']
REGION IV-B (MIMAROPA)
['Palawan', 'Occidental Mindoro', 'Romblon', 'Oriental Mindoro', 'Marinduque']
REGION V (Bicol Region)
['Masbate', 'Camarines Sur', 'Albay', 'Catanduanes', 'Sorsogon', 'Camarines Norte']
REGION VI (Western Visayas)
['Iloilo', 'Aklan', 'Antique', 'Negros Occidental', 'Guimaras', 'Capiz']
REGION VII (Central Visayas)
['Bohol', 'Cebu', 'Negros Oriental', 'Siquijor']
REGION VIII (Eastern Visayas)
['Leyte', 'Northern Samar', 'Samar', 'Biliran', 'Southern Le

In [282]:
all_matched

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [272]:
for possible_province in possible_provinces:
    for city in cities:
        if possible_province.lower() in city.lower():
            ff.append(city)

In [84]:
all_vals = []

In [85]:
df_ = df.loc[df["group"]==1].reset_index()

In [86]:
for region in regions:
    df_ = df.loc[df["group"]==region].reset_index()
    region = df_.iloc[1]["Type of Building and City/Municipality"]
    parse_region(df_)

>>> Parsing  CITY OF MANILA in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF MANDALUYONG in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF MARIKINA in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF PASIG in NATIONAL CAPITAL REGION...
>>> Parsing  QUEZON CITY in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF SAN JUAN in NATIONAL CAPITAL REGION...
>>> Parsing  CALOOCAN CITY in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF MALABON in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF NAVOTAS in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF VALENZUELA in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF LAS PIÑAS in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF MAKATI in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF MUNTINLUPA in NATIONAL CAPITAL REGION...
>>> Parsing  CITY OF PARAÑAQUE in NATIONAL CAPITAL REGION...
>>> Parsing  PASAY CITY in NATIONAL CAPITAL REGION...
>>> Parsing  PATEROS in NATIONAL CAPITAL REGION...
>>> Parsing  TAGUIG CITY in NATIONAL CAPITAL REGION...
>>> Pars

In [87]:
final_df = pd.DataFrame.from_dict(all_vals).drop_duplicates()

## Name matching

In [88]:
def data_cleaner(text):
    # Make lower caps
    text = text.lower()

    # Remove "city"
    text = text.replace("city", "")

    # Remove " of "
    text = text.replace(" of ", "")
    
    # Remove all strings within parentheses
    text = text.split(" (")[0]
    
    # Remove (Capital)
    text = text.replace(" (capital)", "")

    # Remove punctuation marks
    text = re.sub(r"[^\w\s]", "", text)

    # Remove spaces
    text = text.replace(" ", "")

    # Replace compostela valley with Davao de Oro
    text = text.replace("compostelavalley", "davaodeoro")

    # Replace Maguindanao with del Norte/del Sur
    if "maguindanao" in text:
        text = "maguindanao"

    # Replace santo with sto
    text = text.replace("santo", "sto")

    # Replace santa with sta
    text = text.replace("santa", "sta")



    return text

In [90]:
REF_DF = Path(PROJECTROOT,"data","cleaned-datasets","ph-shp-file","ph-shp-file.shp")

In [91]:
ref_df = gp.read_file(REF_DF)

In [92]:
ref_df.head()

,name,city_munic,province,region,coords,geometry
0,Aborlan,Aborlan,Palawan,REGION IV-B (MIMAROPA),"9.4371009, 118.5484168","MULTIPOLYGON (((118.57998 9.37215, 118.57982 9..."
1,Abra De Ilog,Abra De Ilog,Occidental Mindoro,REGION IV-B (MIMAROPA),"13.4437209, 120.7268262","POLYGON ((120.60896 13.35233, 120.60797 13.373..."
2,Abucay,Abucay,Bataan,REGION III (Central Luzon),"14.7213146, 120.5348704","POLYGON ((120.45676 14.69671, 120.45620 14.696..."
3,Abulug,Abulug,Cagayan,REGION II (Cagayan Valley),"18.4434854, 121.4572732","MULTIPOLYGON (((121.40276 18.40896, 121.40276 ..."
4,Abuyog,Abuyog,Leyte,REGION VIII (Eastern Visayas),"10.747102, 125.0114853","POLYGON ((125.04650 10.56751, 125.04588 10.576..."


In [93]:
ref_df["clean_city"] = ref_df["city_munic"].apply(data_cleaner)
ref_df["clean_reg"] = ref_df["region"].apply(data_cleaner)

ref_df["clean_idx"] = (ref_df["clean_city"].astype(str)
    + ", "
    + ref_df["clean_reg"].astype(str))

In [94]:
ref_df.head()

,name,city_munic,province,region,coords,geometry,clean_city,clean_reg,clean_idx
0,Aborlan,Aborlan,Palawan,REGION IV-B (MIMAROPA),"9.4371009, 118.5484168","MULTIPOLYGON (((118.57998 9.37215, 118.57982 9...",aborlan,regionivb,"aborlan, regionivb"
1,Abra De Ilog,Abra De Ilog,Occidental Mindoro,REGION IV-B (MIMAROPA),"13.4437209, 120.7268262","POLYGON ((120.60896 13.35233, 120.60797 13.373...",abradeilog,regionivb,"abradeilog, regionivb"
2,Abucay,Abucay,Bataan,REGION III (Central Luzon),"14.7213146, 120.5348704","POLYGON ((120.45676 14.69671, 120.45620 14.696...",abucay,regioniii,"abucay, regioniii"
3,Abulug,Abulug,Cagayan,REGION II (Cagayan Valley),"18.4434854, 121.4572732","MULTIPOLYGON (((121.40276 18.40896, 121.40276 ...",abulug,regionii,"abulug, regionii"
4,Abuyog,Abuyog,Leyte,REGION VIII (Eastern Visayas),"10.747102, 125.0114853","POLYGON ((125.04650 10.56751, 125.04588 10.576...",abuyog,regionviii,"abuyog, regionviii"


In [95]:
final_df["region_name"]=final_df["region_name"].apply(replace_region)

In [96]:
final_df["clean_city"] = final_df["city"].apply(data_cleaner)
final_df["clean_reg"] = final_df["region_name"].apply(data_cleaner)

final_df["clean_idx"] = (final_df["clean_city"].astype(str)
    + ", "
    + final_df["clean_reg"].astype(str))

Some cities and regions are non-unique. Let's deal with those.

In [203]:
slice = final_df.loc[final_df["bldg_type"]=='Single house']

In [204]:
unique_idx_reg = list(slice["clean_idx"])

In [205]:
newlist = [] # empty list to hold unique elements from the list
duplist = [] # empty list to hold the duplicate elements from the list
for i in unique_idx_reg:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [206]:
duplist

['sanjuan, regioni',
 'sanjose, regioniii',
 'quezon, regioniva',
 'rizal, regionivb',
 'danao, regionvii',
 'valencia, regionvii',
 'magsaysay, regionx',
 'sanfrancisco, regionxiii']

In [222]:
final_df = final_df.loc[final_df["city"]!='BATANGAS']

In [223]:
dup_idx ="sanjuan, regioni"

In [226]:
slice.loc[slice["city"]=="BATANGAS"]

,city,region_name,bldg_type,Occupied Housing Units,Number of Households*,Household Population*,Average Household Size,Ratio of Households to Occupied Housing Units,Ratio of Household Population to Occupied Housing Units,clean_city,clean_reg,clean_idx


In [225]:
ref_df.loc[ref_df["clean_idx"]==dup_idx]

,name,city_munic,province,region,coords,geometry,clean_city,clean_reg,clean_idx
1282,San Juan (IS),San Juan,Ilocos Sur,REGION I (Ilocos Region),"17.7422338, 120.4583071","POLYGON ((120.47397 17.71095, 120.47037 17.712...",sanjuan,regioni,"sanjuan, regioni"
1283,San Juan (LU),San Juan,La Union,REGION I (Ilocos Region),"16.6699328, 120.3385507","POLYGON ((120.43591 16.63852, 120.42805 16.639...",sanjuan,regioni,"sanjuan, regioni"


In [200]:
new_idx = "sanjuan-is, regioni"

In [201]:
final_df.loc[final_df["city"]=="SAN JUAN (LAPOG)"]["clean_idx"] = new_idx

/var/folders/d8/ppdd00vj50s7x6lk402s3p9h0000gn/T/ipykernel_18421/3891042259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[final_df["city"]=="SAN JUAN (LAPOG)"]["clean_idx"] = new_idx


In [227]:
provinces = list(ref_df["province"].unique())
provinces

['Palawan',
 'Occidental Mindoro',
 'Bataan',
 'Cagayan',
 'Leyte',
 'Ilocos Norte',
 'Quezon',
 'Quirino',
 'Pangasinan',
 'Batangas',
 'La Union',
 'Ifugao',
 'Iloilo',
 'Basilan',
 'Sarangani',
 'Cotabato',
 'Laguna',
 'Bohol',
 'Cebu',
 'Romblon',
 'Surigao del Norte',
 'Cavite',
 'Nueva Vizcaya',
 'Nueva Ecija',
 'Isabela',
 'Zamboanga Sibugay',
 'Ilocos Sur',
 'Northern Samar',
 'Samar',
 'Biliran',
 'Misamis Occidental',
 'Aklan',
 'Misamis Oriental',
 'Lanao del Sur',
 'Negros Oriental',
 'Maguindanao del Sur',
 'Southern Leyte',
 'Tarlac',
 'Bulacan',
 'Pampanga',
 'Rizal',
 'Antique',
 'Masbate',
 'Eastern Samar',
 'Davao del Norte',
 'Benguet',
 'Zamboanga del Sur',
 'Camarines Sur',
 'Albay',
 'Oriental Mindoro',
 'Lanao del Norte',
 'Negros Occidental',
 'Zamboanga del Norte',
 'Catanduanes',
 'Davao Oriental',
 'Sultan Kudarat',
 'Kalinga',
 'Aurora',
 'South Cotabato',
 'Abra',
 'Davao del Sur',
 'Sorsogon',
 'Maguindanao del Norte',
 'Mountain Province',
 'Surigao del S

In [238]:
mm_pr =  list(final_df["city"].unique())
mm_pr = [val.split(" (e")[0] for val in mm_pr]
mm_pr.sort()

mm_pr

['ABORLAN',
 'ABRA',
 'ABRA DE ILOG',
 'ABUCAY',
 'ABULUG',
 'ABUYOG',
 'ADAMS',
 'AGDANGAN',
 'AGLIPAY',
 'AGNO',
 'AGONCILLO',
 'AGOO',
 'AGUILAR',
 'AGUINALDO',
 'AGUSAN DEL NORTE',
 'AGUSAN DEL SUR',
 'AGUTAYA',
 'AJUY',
 'AKBAR',
 'AKLAN',
 'AL-BARKA',
 'ALABAT',
 'ALABEL (Capital)',
 'ALAMADA',
 'ALAMINOS',
 'ALANGALANG',
 'ALBAY',
 'ALBUERA',
 'ALBURQUERQUE',
 'ALCALA',
 'ALCANTARA',
 'ALCOY',
 'ALEGRIA',
 'ALEOSAN',
 'ALFONSO',
 'ALFONSO CASTANEDA',
 'ALFONSO LISTA (POTIA)',
 'ALIAGA',
 'ALICIA',
 'ALILEM',
 'ALIMODIAN',
 'ALITAGTAG',
 'ALLACAPAN',
 'ALLEN',
 'ALMAGRO',
 'ALMERIA',
 'ALOGUINSAN',
 'ALORAN',
 'ALTAVAS',
 'ALUBIJID',
 'AMADEO',
 'AMBAGUIO',
 'AMLAN (AYUQUITAN)',
 'AMPATUAN',
 'AMULUNG',
 'ANAHAWAN',
 'ANAO',
 'ANDA',
 'ANGADANAN',
 'ANGAT',
 'ANGELES CITY',
 'ANGONO',
 'ANILAO',
 'ANINI-Y',
 'ANTEQUERA',
 'ANTIPAS',
 'ANTIQUE',
 'APALIT',
 'APARRI',
 'APAYAO',
 'ARACELI',
 'ARAKAN',
 'ARAYAT',
 'ARGAO',
 'ARINGAY',
 'ARITAO',
 'AROROY',
 'ARTECHE',
 'ASINGAN',
 '

In [235]:
ok = [col for col in mm_pr if "(excluding" in col]
ok.sort()
ok

['AGUSAN DEL NORTE (excluding Butuan City)',
 'BENGUET (excluding Baguio City)',
 'DAVAO DEL SUR (excluding Davao City)',
 'LANAO DEL NORTE (excluding Iligan City)',
 'MISAMIS ORIENTAL (excluding Cagayan de Oro City)',
 'NEGROS OCCIDENTAL (excluding Bacolod City)',
 'PALAWAN (excluding Puerto Princesa City)',
 'PAMPANGA (excluding Angeles City)',
 'QUEZON (excluding Lucena City)',
 'SOUTH COTABATO (excluding General Santos City)',
 'ZAMBALES (excluding Olongapo City)',
 'ZAMBOANGA DEL SUR (excluding Zamboanga City)']

In [239]:
not_in_mmpr = [prov for prov in provinces if prov.upper() not in mm_pr]
not_in_mmpr.sort()

not_in_mmpr

['Batangas',
 'Cavite',
 'Cebu',
 'Cotabato',
 'Davao de Oro',
 'Iloilo',
 'Maguindanao del Norte',
 'Maguindanao del Sur',
 'Masbate',
 'Metropolitan Manila',
 'Romblon',
 'Samar',
 'Siquijor',
 'Sorsogon',
 'Tarlac']

In [211]:
final_df.loc[final_df["city"]=="SAN JUAN (LAPOG)"]["clean_idx"]

6006    sanjuan, regioni
6012    sanjuan, regioni
6018    sanjuan, regioni
6024    sanjuan, regioni
6030    sanjuan, regioni
6036    sanjuan, regioni
6042    sanjuan, regioni
Name: clean_idx, dtype: object

In [202]:
ref_df.loc[ref_df["name"]=="San Juan (IS)"]["clean_idx"] = new_idx

/opt/homebrew/Caskroom/miniforge/base/envs/mle/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
